<a href="https://colab.research.google.com/github/OMWANDHOJR/Alx-Movie-recommendation/blob/main/Recommendation_stystem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

alx_movie_recommendation_project_2025_path = kagglehub.competition_download('alx-movie-recommendation-project-2025')

print('Data source import complete.')


100%|██████████| 239M/239M [00:02<00:00, 93.3MB/s]

Extracting files...


Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
movies = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/movies.csv')
imdb = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/imdb_data.csv')
genome_tags= pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/genome_tags.csv')
genome_scores= pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/genome_scores.csv')
train = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/train.csv')
test = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/test.csv')
tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/tags.csv')
links = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2025/links.csv')

movies.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/alx-movie-recommendation-project-2025/movies.csv'

In [5]:
!ls /kaggle/input/alx-movie-recommendation-project-2025/

ls: cannot access '/kaggle/input/alx-movie-recommendation-project-2025/': No such file or directory


In [8]:
!pip install kagglehub
import kagglehub
kagglehub.login()

# Download the dataset
alx_movie_recommendation_project_2025_path = kagglehub.competition_download('alx-movie-recommendation-project-2025')

import os
import pandas as pd

# Assuming the CSV files are directly within the downloaded directory
data_dir = alx_movie_recommendation_project_2025_path

# Update the file paths for reading the CSV files
movies = pd.read_csv(os.path.join(data_dir, 'movies.csv'))
imdb = pd.read_csv(os.path.join(data_dir, 'imdb_data.csv'))
genome_tags = pd.read_csv(os.path.join(data_dir, 'genome_tags.csv'))
genome_scores = pd.read_csv(os.path.join(data_dir, 'genome_scores.csv'))
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
tags = pd.read_csv(os.path.join(data_dir, 'tags.csv'))
links = pd.read_csv(os.path.join(data_dir, 'links.csv'))

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Kaggle credentials set.
Kaggle credentials successfully validated.


In [9]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [10]:
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [11]:
test.isnull().sum()

,0
userId,0
movieId,0


In [12]:


# Calculate average rating for each movie
movie_ratings = train.groupby('movieId')['rating'].mean()
# Calculate average rating for each user
user_ratings = train.groupby('userId')['rating'].mean()

# Create dictionaries to store averages for faster lookup
movie_avg_ratings = movie_ratings.to_dict()
user_avg_ratings = user_ratings.to_dict()

# Calculate overall average rating
overall_avg_rating = train['rating'].mean()

# Function to predict ratings for the test data
def predict_rating(row):
    movie_id = row['movieId']
    user_id = row['userId']

    # Get movie and user average ratings
    movie_avg = movie_avg_ratings.get(movie_id, overall_avg_rating)
    user_avg = user_avg_ratings.get(user_id, overall_avg_rating)

    # Predict rating using user and movie biases
    predicted_rating = overall_avg_rating + (user_avg - overall_avg_rating) + (movie_avg - overall_avg_rating)

    return predicted_rating

# Predict ratings for the test data
test['predicted_rating'] = test.apply(predict_rating, axis=1)

# Function to recommend movies for a user based on predicted ratings
def recommend_movies(user_id, top_n=10):
    # Filter test data for the given user
    user_predictions = test[test['userId'] == user_id]
    # Sort by predicted rating in descending order
    sorted_predictions = user_predictions.sort_values('predicted_rating', ascending=False)
    # Get the top N movie recommendations
    top_movies = sorted_predictions.head(top_n)['movieId'].tolist()
    return top_movies

# Example usage: Recommend top 10 movies for user ID 1
recommended_movies = recommend_movies(1)
print(f"Recommended movies for user ID 1: {recommended_movies}")

# Save predictions to a CSV file
# test[['userId', 'movieId', 'predicted_rating']].to_csv('predictions.csv', index=False)

Recommended movies for user ID 1: [4144, 8786, 6711, 8405, 5767, 2011, 7318]


In [13]:


# ... (Previous code for calculating averages and predicting ratings remains the same) ...

# Create the 'Id' column in the test DataFrame
test['Id'] = test['userId'].astype(str) + '_' + test['movieId'].astype(str)

# Select the required columns and save to CSV
submission_df = test[['Id', 'predicted_rating']]
submission_df.columns = ['Id', 'rating']  # Rename 'predicted_rating' to 'rating'
submission_df.to_csv('/kaggle/submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
